In [1]:
import boto3
import sagemaker
from sagemaker import Model
import os
import json

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
sagemaker_session = sagemaker.Session()

role = sagemaker.get_execution_role()
print(f"SageMaker Role ARN: {role}")

SageMaker Role ARN: arn:aws:iam::796932308591:role/service-role/SageMaker-ExecutionRole-20250214T145019


In [3]:
# import boto3
# import os
# import tarfile
# import shutil

# # Initialize S3 client
# s3 = boto3.client('s3')

# # Define S3 bucket and file paths
# bucket = 'mitrailabs-personaclassification'
# model_prefix = 'risk_prediction/Intermediate_states/'
# model_files = [
#     'nimputer.joblib',  
#     'cimputer.joblib',  
#     'label_encoders.joblib',  # Keeping as pickle  
#     'inference_preprocessor.joblib',  
#     'before_feature.json'  
# ]

# model_files_s3_keys = [f"{model_prefix}{file}" for file in model_files]

# # Add the XGBoost model (now using .joblib)
# xgb_model_s3_key = 'risk_prediction/models/xgb_model.joblib'
# model_files_s3_keys.append(xgb_model_s3_key)

# # Define the local artifacts directory
# artifacts_dir = 'artifacts'
# os.makedirs(artifacts_dir, exist_ok=True)  # Create the directory if it doesn't exist

# # Download all files from S3
# for s3_key in model_files_s3_keys:
#     file_name = os.path.basename(s3_key)
#     local_path = os.path.join(artifacts_dir, file_name)
#     s3.download_file(bucket, s3_key, local_path)
#     print(f"Downloaded {s3_key} to {local_path}")

# # Copy the inference script to the artifacts directory
# inference_script_path = os.path.join(artifacts_dir, 'inference.py')
# if os.path.exists('inference.py'):
#     shutil.copy('inference.py', inference_script_path)
#     print(f"Copied inference.py to {artifacts_dir}")
# else:
#     print("inference.py does not exist in the current directory.")
#     # Handle missing inference.py as needed (e.g., raise an error or create a placeholder)

# # Copy the requirements.txt file to the artifacts directory
# requirements_file_path = 'requirements.txt'  # Path to your requirements.txt
# if os.path.exists(requirements_file_path):
#     shutil.copy(requirements_file_path, artifacts_dir)
#     print(f"Copied {requirements_file_path} to {artifacts_dir}")
# else:
#     print(f"{requirements_file_path} does not exist in the current directory.")
#     # Handle missing requirements.txt as needed (e.g., create an empty file, raise an error)

# # Create a .tar.gz file from the artifacts directory
# tar_gz_path = os.path.join(artifacts_dir, 'model.tar.gz')
# with tarfile.open(tar_gz_path, 'w:gz') as tar:
#     for file_name in os.listdir(artifacts_dir):
#         file_path = os.path.join(artifacts_dir, file_name)
#         tar.add(file_path, arcname=file_name)
# print(f"Created {tar_gz_path}")

# # Upload the .tar.gz file to S3
# s3_key = 'risk_prediction/models/model.tar.gz'
# s3.upload_file(tar_gz_path, bucket, s3_key)
# print(f"Uploaded {tar_gz_path} to s3://{bucket}/{s3_key}")

In [51]:
import boto3
import os
import tarfile
import shutil

# Initialize S3 client
s3 = boto3.client('s3')

# Define S3 bucket and file paths
bucket = 'mitrailabs-personaclassification'
model_prefix = 'risk_prediction/Intermediate_states/'
model_files = [
    'nimputer.joblib',  
    'cimputer.joblib',  
    'label_encoders.joblib',  # Keeping as pickle  
    # 'inference_preprocessor.joblib',  
    'before_feature.json'  
]

model_files_s3_keys = [f"{model_prefix}{file}" for file in model_files]

# Add the XGBoost model (now using .joblib)
xgb_model_s3_key = 'risk_prediction/models/xgb_model.json'
model_files_s3_keys.append(xgb_model_s3_key)

# Define the local artifacts directory
artifacts_dir = 'artifacts'
os.makedirs(artifacts_dir, exist_ok=True)  # Create the directory if it doesn't exist

# Download all files from S3
for s3_key in model_files_s3_keys:
    file_name = os.path.basename(s3_key)
    local_path = os.path.join(artifacts_dir, file_name)
    s3.download_file(bucket, s3_key, local_path)
    print(f"Downloaded {s3_key} to {local_path}")

# Create a 'code' directory inside the artifacts directory
code_dir = os.path.join(artifacts_dir, 'code')
os.makedirs(code_dir, exist_ok=True)

# Copy the inference script to the 'code' directory
inference_script_path = os.path.join(code_dir, 'inference.py')
if os.path.exists('inference.py'):
    shutil.copy('inference.py', inference_script_path)
    print(f"Copied inference.py to {code_dir}")
else:
    print("inference.py does not exist in the current directory.")
    # Handle missing inference.py as needed (e.g., raise an error or create a placeholder)

# Copy the requirements.txt file to the 'code' directory
requirements_file_path = 'requirements.txt'  # Path to your requirements.txt
if os.path.exists(requirements_file_path):
    shutil.copy(requirements_file_path, code_dir)
    print(f"Copied {requirements_file_path} to {code_dir}")
else:
    print(f"{requirements_file_path} does not exist in the current directory.")
    # Handle missing requirements.txt as needed (e.g., create an empty file, raise an error)

# Create a .tar.gz file from the artifacts directory
tar_gz_path = os.path.join(artifacts_dir, 'model.tar.gz')
with tarfile.open(tar_gz_path, 'w:gz') as tar:
    # Add model files directly to the root of the tar
    for file_name in os.listdir(artifacts_dir):
        if file_name != 'code' and file_name != 'model.tar.gz':
            file_path = os.path.join(artifacts_dir, file_name)
            tar.add(file_path, arcname=file_name)
    
    # Add the 'code' directory to the tar
    for file_name in os.listdir(code_dir):
        file_path = os.path.join(code_dir, file_name)
        tar.add(file_path, arcname=os.path.join('code', file_name))

print(f"Created {tar_gz_path}")

# Upload the .tar.gz file to S3
s3_key = 'risk_prediction/models/model.tar.gz'
s3.upload_file(tar_gz_path, bucket, s3_key)
print(f"Uploaded {tar_gz_path} to s3://{bucket}/{s3_key}")

Downloaded risk_prediction/Intermediate_states/nimputer.joblib to artifacts/nimputer.joblib
Downloaded risk_prediction/Intermediate_states/cimputer.joblib to artifacts/cimputer.joblib
Downloaded risk_prediction/Intermediate_states/label_encoders.joblib to artifacts/label_encoders.joblib
Downloaded risk_prediction/Intermediate_states/before_feature.json to artifacts/before_feature.json
Downloaded risk_prediction/models/xgb_model.json to artifacts/xgb_model.json
Copied inference.py to artifacts/code
Copied requirements.txt to artifacts/code
Created artifacts/model.tar.gz
Uploaded artifacts/model.tar.gz to s3://mitrailabs-personaclassification/risk_prediction/models/model.tar.gz


In [3]:
from sagemaker.xgboost.model import XGBoostModel
from sagemaker.xgboost import XGBoost
from sagemaker.estimator import Estimator
from sagemaker.model import Model
# import sklearn
# import xgboost

# with open("requirements.txt", "w") as f:
#     f.write(f"scikit-learn=={sklearn.__version__}\n")
#     f.write(f"xgboost=={xgboost.__version__}\n") # Add xgboost
#     f.write("pandas\n") # Add other dependencies
#     f.write("numpy\n")

target_bucket = 'mitrailabs-personaclassification'
model_path = 'risk_prediction/models/model.tar.gz'
model_s3_path = f's3://{target_bucket}/{model_path}'

model = XGBoostModel(
    model_data=model_s3_path,
    role=role,
    entry_point='inference.py',
    framework_version='1.7-1',  # Matching closer to your 2.1.4
    py_version='py3',
    # dependencies=['requirements2.txt'],
    sagemaker_session=sagemaker_session
)

# model = XGBoost(
#     model_data=model_s3_path,
#     role=role,
#     entry_point='inference.py',
#     framework_version='1.5-1',  # Matching closer to your 2.1.4
#     py_version='py3',
#     sagemaker_session=sagemaker_session,
#     requirements_file = 'requirements.txt'
# )

print("done")

done


In [ ]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium',  # Cost-effective choice
    endpoint_name='customer-creditrisk-V1',
)

-------!

In [7]:
import json
import numpy as np

user_input = {
    'score_3': 800,  # High credit score
    'score_4': np.nan,  # Missing value
    'score_5': 0.850,  # High probability score
    'score_6': 98,  # High score
    'risk_rate': 0.10,  # Low risk rate
    'last_amount_borrowed': 200,  # Low amount borrowed
    'last_borrowed_in_months': 36,  # Long time since last borrowing
    'credit_limit': 10,  # High credit limit
    'income': np.nan,  # High income
    'ok_since': 60,  # Long time since last issue
    'n_bankruptcies': 0.0,  # No bankruptcies
    'n_defaulted_loans': 0,  # No defaulted loans
    'n_accounts': 10.0,  # Moderate number of accounts
    'n_issues': 1.0,  # Few issues
    'external_data_provider_credit_checks_last_year': 1.0,  # Few credit checks
    'external_data_provider_email_seen_before': 10,  # Email seen many times (trusted)
    'reported_income': 120000,  # High reported income
    'application_time_in_funnel': 300,  # Short application time
    'external_data_provider_credit_checks_last_month': 0.0,  # No recent credit checks
    'external_data_provider_fraud_score': 100,  # Low fraud score
    'shipping_state': np.nan,  # Missing value
    'facebook_profile': True,  # Has a Facebook profile
    'state': np.nan,  # Missing value
    'score_1': '1Rk8w4Ucd5yR3KcqZzLdow==',  # Example encrypted score
    'score_2': np.nan,  # Missing value
    'real_state': np.nan  # Missing value
}

from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

endpoint_name = "customer-creditrisk-V1"

# Create a predictor object
predictor = Predictor(
    endpoint_name=endpoint_name,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer()
)


input_data = json.dumps(user_input)
response = predictor.predict(input_data)
print("\nPrediction Response:")
print(response)


Prediction Response:
{'prediction': 3, 'reasoning': 'Medium Risk \n Borrowers in this cluster have a moderate default rate (0.117) and show mixed borrowing behavior. They have taken very small loans (last_amount_borrowed: 0.008) and have a low debt-to-income ratio (0.007). Their credit utilization (0.001) is minimal, but their risk scores (score_1, score_2) show more variation, suggesting the need for closer monitoring. Their behavior is generally positive but requires caution.', 'prediction_proba': [0.038472529500722885, 0.038463044911623, 0.03846452385187149, 0.8460456728935242, 0.038554225116968155]}


In [78]:
import json
prediction = [[1, 2, 3, 4]]
reasoning = {'1':1, '2':2, '3':"fnejfn fjenfijn", '4':4}
result = {
            "prediction": np.argmax((prediction[0])).tolist(),
            "reasoning": reasoning[str(np.argmax((prediction[0])))],
            "prediction_proba": prediction[0]
        }

json.dumps(result)

'{"prediction": 3, "reasoning": "fnejfn fjenfijn", "prediction_proba": [1, 2, 3, 4]}'

In [8]:
import boto3
import time
from tqdm import tqdm

def delete_all_endpoints(name_filter=None):
    sm_client = boto3.client('sagemaker')

    print("Listing endpoints...")
    endpoints = sm_client.list_endpoints()['Endpoints']

    if name_filter:
        endpoints = [ep for ep in endpoints if name_filter in ep['EndpointName']]

    if not endpoints:
        print("No endpoints found to delete.")
        return

    print(f"Found {len(endpoints)} endpoints to delete.")

    for endpoint in endpoints:
        endpoint_name = endpoint['EndpointName']
        print(f"\nDeleting endpoint: {endpoint_name}")

        try:
            sm_client.delete_endpoint(EndpointName=endpoint_name)
            print(f"Waiting for endpoint {endpoint_name} to be deleted...")

            # Use a waiter with exponential backoff
            waiter = sm_client.get_waiter('endpoint_deleted')
            waiter.wait(EndpointName=endpoint_name, 
                        WaiterConfig={'Delay': 5, 'MaxAttempts': 30})

            print(f"Endpoint {endpoint_name} deleted successfully.")

        except sm_client.exceptions.ResourceNotFound:  # Catch ResourceNotFound
            print(f"Endpoint {endpoint_name} not found. It may have already been deleted.")

        except Exception as e:
            print(f"Error deleting endpoint {endpoint_name}: {str(e)}")

        # Delete endpoint config (optional)
        try:
            sm_client.delete_endpoint_config(EndpointConfigName=endpoint_name)
            print(f"Endpoint configuration {endpoint_name} deleted.")
        except Exception as e:
            print(f"Could not delete endpoint config {endpoint_name}: {str(e)}")

delete_all_endpoints()

Listing endpoints...
Found 1 endpoints to delete.

Deleting endpoint: customer-creditrisk-V1
Waiting for endpoint customer-creditrisk-V1 to be deleted...
Endpoint customer-creditrisk-V1 deleted successfully.
Endpoint configuration customer-creditrisk-V1 deleted.
